In [ ]:
!pip install mysql-connector-python

In [4]:
import mysql.connector
import pandas as pd
import numpy as np

In [7]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",          # Change to your MySQL host
    user="root",      # Change to your MySQL username
    password="951Kdroot@12",  # Change to your MySQL password
    database="MLhack"   # Change to your database name
)
cursor = conn.cursor()

In [9]:
# Step 1: Create the `customers` table
create_table_query = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY AUTO_INCREMENT,
    age INT,
    tenure INT,
    monthly_usage FLOAT,
    complaints INT,
    returns INT,
    emails_opened INT,
    daily_logins INT,
    sensor_triggers INT
);
"""
cursor.execute(create_table_query)
conn.commit()

In [11]:
# Step 2: Generate Synthetic Data
num_customers = 1000  # Number of customers to generate


In [13]:
# Generate random values for each attribute
customer_data = pd.DataFrame({
    'age': np.random.randint(18, 70, num_customers),  # Age between 18 and 70
    'tenure': np.random.randint(1, 60, num_customers),  # Tenure in months
    'monthly_usage': np.random.uniform(10, 500, num_customers),  # Monthly usage in arbitrary units
    'complaints': np.random.randint(0, 5, num_customers),  # Number of complaints
    'returns': np.random.randint(0, 3, num_customers),  # Number of returns
    'emails_opened': np.random.randint(0, 10, num_customers),  # Emails opened in the past month
    'daily_logins': np.random.randint(0, 5, num_customers),  # Average daily logins
    'sensor_triggers': np.random.randint(0, 10, num_customers)  # Sensor triggers in the past month
})

In [15]:
# Step 3: Insert Data into the MySQL Table
insert_query = """
INSERT INTO customers (age, tenure, monthly_usage, complaints, returns, emails_opened, daily_logins, sensor_triggers)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Iterate over the generated DataFrame and insert each row into the database
for _, row in customer_data.iterrows():
    cursor.execute(insert_query, tuple(row))

conn.commit()
print(f"{num_customers} customer records inserted into the database.")

# Close the database connection
cursor.close()
conn.close()

1000 customer records inserted into the database.


Insert data in events table

Lets go

In [18]:
import pandas as pd
import numpy as np
import datetime
import mysql.connector

# MySQL connection setup
connection = mysql.connector.connect(
    host="localhost",  # Change to your host
    user="root",       # Change to your username
    password="951Kdroot@12",  # Change to your password
    database="MLhack"  # Change to your database name
)
cursor = connection.cursor()

# Number of events for streaming
num_events = 5000
start_date = datetime.datetime(2024, 1, 1)

# Generate event data
event_data = {
    'Customer_ID': np.random.randint(1, 1001, num_events),  # Ensures IDs are between 1 and 1000
    'Event_Timestamp': [start_date + datetime.timedelta(days=np.random.randint(0, 180)) for _ in range(num_events)],
    'Event_Type': np.random.choice(['login', 'sensor_trigger', 'email_open', 'Returns', 'Complaints'], num_events),
}

# Logical Event Values
def generate_event_value(row):
    if row['Event_Type'] in ['login', 'email_open']:
        return np.random.randint(1, 5)  # Lower range for simple actions
    elif row['Event_Type'] == 'sensor_trigger':
        return np.random.randint(1, 10)  # Medium range for IoT events
    elif row['Event_Type'] == 'Returns':
        return np.random.randint(1, 3)  # Binary-like data
    elif row['Event_Type'] == 'Complaints':
        return np.random.randint(1, 8)  # Higher range for complaints
    return 1  # Fallback value (though unlikely to be used)

# Create DataFrame
event_df = pd.DataFrame(event_data)

# Add Event_Value based on logic
event_df['Event_Value'] = event_df.apply(generate_event_value, axis=1)

# Sort by Event_Timestamp to mimic realistic streaming
event_df = event_df.sort_values(by='Event_Timestamp')

# Insert data directly into MySQL
insert_query = """
    INSERT INTO events (Customer_ID, Event_Timestamp, Event_Type, Event_Value)
    VALUES (%s, %s, %s, %s)
"""

# Prepare data for bulk insert
event_rows = [
    (
        int(row['Customer_ID']),
        row['Event_Timestamp'].strftime('%Y-%m-%d %H:%M:%S'),
        row['Event_Type'],
        int(row['Event_Value'])
    )
    for _, row in event_df.iterrows()
]

try:
    # Bulk insert into MySQL
    cursor.executemany(insert_query, event_rows)
    connection.commit()
    print(f"Successfully inserted {len(event_rows)} rows into the 'events' table.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    connection.rollback()

# Close connection
cursor.close()
connection.close()


Successfully inserted 5000 rows into the 'events' table.
